In [ ]:
import polars as pl
import xarray as xr
import plotly.express as px
import numpy as np

In [ ]:
weather_forecast = pl.scan_parquet("data/met_forecast.parquet")
weather_nowcast = pl.scan_parquet("data/met_nowcast.parquet")
windpower = pl.scan_parquet("data/wind_power_per_bidzone.parquet").rename(
    {"__index_level_0__": "time"}
)
windparks = pl.scan_csv("data/windparks_bidzone.csv", try_parse_dates=True).filter(
    pl.col("eic_code") == pl.col("eic_code").first().over("substation_name")
)

In [ ]:
windparks.select(pl.col("prod_start_new").max()).collect()

In [ ]:
bid_zone = "ELSPOT NO3"
ensemble_member = 0
bid_zone_weather = (
    weather_forecast.join(windparks, left_on="sid", right_on="substation_name")
    .filter(
        pl.col("bidding_area") == bid_zone,
        pl.col("time_ref") > pl.col("prod_start_new"),
    )
    .select(
        "sid",
        "time_ref",
        "time",
        "lt",
        "operating_power_max",
        f"ws10m_{ensemble_member:02d}",
        f"t2m_{ensemble_member:02d}",
        f"rh2m_{ensemble_member:02d}",
        f"mslp_{ensemble_member:02d}",
        f"g10m_{ensemble_member:02d}",
    )
)
bid_zone_weather.tail(10).collect()

In [ ]:
bid_zone_weather.collect_schema().names()

In [ ]:
px.histogram(windparks.collect(), "prod_start_new")

In [ ]:
timestamps = (
    bid_zone_weather.select(pl.col("time_ref").unique()).collect().to_series().sort()
)
parks = bid_zone_weather.select(pl.col("sid").unique()).collect().to_series().sort()
variables = [
    "lt",
    "operating_power_max",
    "ws10m_00",
    "t2m_00",
    "rh2m_00",
    "mslp_00",
    "g10m_00",
]
parks

In [ ]:
values = np.zeros((len(parks), len(timestamps), 62, len(variables)), dtype=np.float32)
for i, park in enumerate(parks):
    for j, ts in enumerate(timestamps):
        tmp = bid_zone_weather.filter(
            pl.col("sid") == park, pl.col("time_ref") == ts
        ).collect()
        break
        # values[i, j] = bid_zone_weather.filter(pl.col("sid")==park, pl.col("time_ref") == ts).select(variables).collect().to_numpy()

In [ ]:
tmp

In [ ]:
ws = xr.DataArray(
    aiport_distances_raster.astype(np.float32),
    coords={"y": y, "x": x},
    dims=("y", "x"),
    name="aiport_distance",
)